    1. Implémentation manuelle de l’inverse modulaire (Algorithme d’Euclide étendu)

In [ ]:
def inverse_modulaire(a, m):
    r0, r1 = a, m
    x0, x1 = 1, 0
    y0, y1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1

    if r0 != 1:
        raise ValueError("L'inverse modulaire n'existe pas (a et m ne sont pas premiers entre eux)")

    return x0 % m 

# Exemple de test
a = 17
m = 3120
print("Inverse modulaire de", a, "mod", m, "est :", inverse_modulaire(a, m))


Inverse modulaire de 17 mod 3120 est : 2753


    2. Calcul de l’inverse modulaire avec pow() et sympy.mod_inverse

                Méthode 1 : Avec la fonction pow() (intégrée à Python)

In [43]:
# pow(a, -1, m) donne l'inverse modulaire de a mod m
a = 17
m = 3120
try:
    inverse = pow(a, -1, m)
    print("Inverse modulaire avec pow():", inverse)
except ValueError:
    print("L'inverse modulaire n'existe pas")


Inverse modulaire avec pow(): 2753


                Méthode 2 : Avec sympy.mod_inverse

In [44]:
from sympy import mod_inverse

a = 17
m = 3120
try:
    inverse = mod_inverse(a, m)
    print("Inverse modulaire avec sympy.mod_inverse:", inverse)
except ValueError:
    print("L'inverse modulaire n'existe pas")


Inverse modulaire avec sympy.mod_inverse: 2753


    3. Implémentation de SHA-1

                Étape 1 : Padding du message (pad_message)

In [45]:
def pad_message(message: bytes) -> bytes:
    original_length = len(message) * 8  # longueur en bits
    message += b'\x80'  # Ajout du bit '1' suivi de 7 bits '0'

    while (len(message) * 8) % 512 != 448:
        message += b'\x00'

    message += original_length.to_bytes(8, byteorder='big')  # Longueur sur 64 bits
    return message


                Étape 2 : Initialisation des registres

In [46]:
H0 = 0x67452301
H1 = 0xEFCDAB89
H2 = 0x98BADCFE
H3 = 0x10325476
H4 = 0xC3D2E1F0

                Étape 3 : Traitement des blocs

Fonction de rotation à gauche

In [47]:
def left_rotate(n, b):
    return ((n << b) | (n >> (32 - b))) & 0xFFFFFFFF

Fonction de traitement d’un bloc (512 bits)

In [48]:
def process_block(block, H):
    W = [int.from_bytes(block[i:i+4], 'big') for i in range(0, 64, 4)]

    for i in range(16, 80):
        W.append(left_rotate(W[i-3] ^ W[i-8] ^ W[i-14] ^ W[i-16], 1))

    a, b, c, d, e = H

    for i in range(80):
        if 0 <= i <= 19:
            f = (b & c) | ((~b) & d)
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        else:
            f = b ^ c ^ d
            k = 0xCA62C1D6

        temp = (left_rotate(a, 5) + f + e + k + W[i]) & 0xFFFFFFFF
        e = d
        d = c
        c = left_rotate(b, 30)
        b = a
        a = temp

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF

    return H


                Étape 4 : Fonction principale de hachage SHA-1

In [49]:
def sha1(message: bytes) -> str:
    message = pad_message(message)
    H = [H0, H1, H2, H3, H4]

    for i in range(0, len(message), 64):
        block = message[i:i+64]
        H = process_block(block, H)

    return ''.join(f'{h:08x}' for h in H)


                Étape 5 : Test

In [50]:
if __name__ == "__main__":
    msg = b"hello"
    print("SHA-1 custom:", sha1(msg))

SHA-1 custom: aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d


    4. Comparaison avec hashlib.sha1()

                Code de comparaison :

In [51]:
import hashlib

# Message à tester
message = b"hello"

# Résultat de notre implémentation
mon_hash = sha1(message)

# Résultat avec hashlib
hashlib_hash = hashlib.sha1(message).hexdigest()

# Affichage
print("Mon SHA-1       :", mon_hash)
print("hashlib SHA-1   :", hashlib_hash)

# Vérification
print("Les deux résultats sont-ils identiques ?", mon_hash == hashlib_hash)


Mon SHA-1       : aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
hashlib SHA-1   : aaf4c61ddcc5e8a2dabede0f3b482cd9aea9434d
Les deux résultats sont-ils identiques ? True


    5. Implémentation de SHA-256 (simplifiée mais fidèle)

                Étape 1 : Padding du message (SHA-256)

In [52]:
def pad_message_sha256(message: bytes) -> bytes:
    original_length = len(message) * 8  # en bits
    message += b'\x80'

    while (len(message) * 8) % 512 != 448:
        message += b'\x00'

    message += original_length.to_bytes(8, byteorder='big')
    return message

                 Étape 2 : Initialisation des registres SHA-256

In [53]:
# Valeurs initiales SHA-256 (32 bits)
H256 = [
    0x6a09e667,
    0xbb67ae85,
    0x3c6ef372,
    0xa54ff53a,
    0x510e527f,
    0x9b05688c,
    0x1f83d9ab,
    0x5be0cd19
]

                Étape 3 : Constantes K

In [54]:
K256 = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]


                Étape 4 : Fonctions de SHA-256

In [55]:
def right_rotate(value, bits):
    return ((value >> bits) | (value << (32 - bits))) & 0xFFFFFFFF

                Étape 5 : Traitement d’un bloc de 512 bits

In [56]:
def process_block_sha256(block, H):
    W = [int.from_bytes(block[i:i+4], 'big') for i in range(0, 64, 4)]

    for i in range(16, 64):
        s0 = right_rotate(W[i-15], 7) ^ right_rotate(W[i-15], 18) ^ (W[i-15] >> 3)
        s1 = right_rotate(W[i-2], 17) ^ right_rotate(W[i-2], 19) ^ (W[i-2] >> 10)
        W.append((W[i-16] + s0 + W[i-7] + s1) & 0xFFFFFFFF)

    a, b, c, d, e, f, g, h = H

    for i in range(64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ ((~e) & g)
        temp1 = (h + S1 + ch + K256[i] + W[i]) & 0xFFFFFFFF
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xFFFFFFFF

        h = g
        g = f
        f = e
        e = (d + temp1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xFFFFFFFF

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF
    H[5] = (H[5] + f) & 0xFFFFFFFF
    H[6] = (H[6] + g) & 0xFFFFFFFF
    H[7] = (H[7] + h) & 0xFFFFFFFF

    return H

                Étape 6 : SHA-256 complet

In [57]:
def sha256(message: bytes) -> str:
    message = pad_message_sha256(message)
    H = H256.copy()

    for i in range(0, len(message), 64):
        block = message[i:i+64]
        H = process_block_sha256(block, H)

    return ''.join(f'{h:08x}' for h in H)

                Test final

In [58]:
if __name__ == "__main__":
    msg = b"hello"
    print("SHA-256 custom:", sha256(msg))


SHA-256 custom: 2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824


     6. Afficher chaque étape du calcul dans SHA-1

On va modifier la fonction process_block() pour imprimer les étapes de transformation :
chaque itération, avec les valeurs de a, b, c, d, e, etc.

                Version modifiée de process_block() (SHA-1) avec print() :

In [59]:
def process_block_verbose(block, H):
    W = [int.from_bytes(block[i:i+4], 'big') for i in range(0, 64, 4)]

    for i in range(16, 80):
        W.append(left_rotate(W[i-3] ^ W[i-8] ^ W[i-14] ^ W[i-16], 1))

    a, b, c, d, e = H

    print("\n--- Début du traitement d'un bloc ---")
    for i in range(80):
        if 0 <= i <= 19:
            f = (b & c) | ((~b) & d)
            k = 0x5A827999
        elif 20 <= i <= 39:
            f = b ^ c ^ d
            k = 0x6ED9EBA1
        elif 40 <= i <= 59:
            f = (b & c) | (b & d) | (c & d)
            k = 0x8F1BBCDC
        else:
            f = b ^ c ^ d
            k = 0xCA62C1D6

        temp = (left_rotate(a, 5) + f + e + k + W[i]) & 0xFFFFFFFF
        e = d
        d = c
        c = left_rotate(b, 30)
        b = a
        a = temp

        print(f"Étape {i:02}: a={a:08x}, b={b:08x}, c={c:08x}, d={d:08x}, e={e:08x}")

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF

    print("--- Fin du bloc ---\n")
    return H


                Utilisation dans la fonction sha1_verbose :

In [60]:
def sha1_verbose(message: bytes) -> str:
    message = pad_message(message)
    H = [H0, H1, H2, H3, H4]

    for i in range(0, len(message), 64):
        block = message[i:i+64]
        H = process_block_verbose(block, H)

    return ''.join(f'{h:08x}' for h in H)


                Test :

In [61]:
if __name__ == "__main__":
    sha1_verbose(b"abc")


--- Début du traitement d'un bloc ---
Étape 00: a=0116fc33, b=67452301, c=7bf36ae2, d=98badcfe, e=10325476
Étape 01: a=8990536d, b=0116fc33, c=59d148c0, d=7bf36ae2, e=98badcfe
Étape 02: a=a1390f08, b=8990536d, c=c045bf0c, d=59d148c0, e=7bf36ae2
Étape 03: a=cdd8e11b, b=a1390f08, c=626414db, d=c045bf0c, e=59d148c0
Étape 04: a=cfd499de, b=cdd8e11b, c=284e43c2, d=626414db, e=c045bf0c
Étape 05: a=3fc7ca40, b=cfd499de, c=f3763846, d=284e43c2, e=626414db
Étape 06: a=993e30c1, b=3fc7ca40, c=b3f52677, d=f3763846, e=284e43c2
Étape 07: a=9e8c07d4, b=993e30c1, c=0ff1f290, d=b3f52677, e=f3763846
Étape 08: a=4b6ae328, b=9e8c07d4, c=664f8c30, d=0ff1f290, e=b3f52677
Étape 09: a=8351f929, b=4b6ae328, c=27a301f5, d=664f8c30, e=0ff1f290
Étape 10: a=fbda9e89, b=8351f929, c=12dab8ca, d=27a301f5, e=664f8c30
Étape 11: a=63188fe4, b=fbda9e89, c=60d47e4a, d=12dab8ca, e=27a301f5
Étape 12: a=4607b664, b=63188fe4, c=7ef6a7a2, d=60d47e4a, e=12dab8ca
Étape 13: a=9128f695, b=4607b664, c=18c623f9, d=7ef6a7a2, e=60d4

    Affichage des étapes internes de SHA-256

                Version process_block_sha256_verbose() :

In [62]:
def process_block_sha256_verbose(block, H):
    W = [int.from_bytes(block[i:i+4], 'big') for i in range(0, 64, 4)]

    for i in range(16, 64):
        s0 = right_rotate(W[i-15], 7) ^ right_rotate(W[i-15], 18) ^ (W[i-15] >> 3)
        s1 = right_rotate(W[i-2], 17) ^ right_rotate(W[i-2], 19) ^ (W[i-2] >> 10)
        W.append((W[i-16] + s0 + W[i-7] + s1) & 0xFFFFFFFF)

    a, b, c, d, e, f, g, h = H

    print("\n--- Début du traitement d’un bloc SHA-256 ---")
    for i in range(64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ ((~e) & g)
        temp1 = (h + S1 + ch + K256[i] + W[i]) & 0xFFFFFFFF
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xFFFFFFFF

        h = g
        g = f
        f = e
        e = (d + temp1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xFFFFFFFF

        print(f"Étape {i:02}: a={a:08x} b={b:08x} c={c:08x} d={d:08x} e={e:08x} f={f:08x} g={g:08x} h={h:08x}")

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF
    H[5] = (H[5] + f) & 0xFFFFFFFF
    H[6] = (H[6] + g) & 0xFFFFFFFF
    H[7] = (H[7] + h) & 0xFFFFFFFF

    print("--- Fin du bloc ---\n")
    return H


                Fonction sha256_verbose() complète :

In [63]:
def sha256_verbose(message: bytes) -> str:
    message = pad_message_sha256(message)
    H = H256.copy()

    for i in range(0, len(message), 64):
        block = message[i:i+64]
        H = process_block_sha256_verbose(block, H)

    return ''.join(f'{h:08x}' for h in H)


                Test :

In [64]:
if __name__ == "__main__":
    sha256_verbose(b"abc")


--- Début du traitement d’un bloc SHA-256 ---
Étape 00: a=5d6aebcd b=6a09e667 c=bb67ae85 d=3c6ef372 e=fa2a4622 f=510e527f g=9b05688c h=1f83d9ab
Étape 01: a=5a6ad9ad b=5d6aebcd c=6a09e667 d=bb67ae85 e=78ce7989 f=fa2a4622 g=510e527f h=9b05688c
Étape 02: a=c8c347a7 b=5a6ad9ad c=5d6aebcd d=6a09e667 e=f92939eb f=78ce7989 g=fa2a4622 h=510e527f
Étape 03: a=d550f666 b=c8c347a7 c=5a6ad9ad d=5d6aebcd e=24e00850 f=f92939eb g=78ce7989 h=fa2a4622
Étape 04: a=04409a6a b=d550f666 c=c8c347a7 d=5a6ad9ad e=43ada245 f=24e00850 g=f92939eb h=78ce7989
Étape 05: a=2b4209f5 b=04409a6a c=d550f666 d=c8c347a7 e=714260ad f=43ada245 g=24e00850 h=f92939eb
Étape 06: a=e5030380 b=2b4209f5 c=04409a6a d=d550f666 e=9b27a401 f=714260ad g=43ada245 h=24e00850
Étape 07: a=85a07b5f b=e5030380 c=2b4209f5 d=04409a6a e=0c657a79 f=9b27a401 g=714260ad h=43ada245
Étape 08: a=8e04ecb9 b=85a07b5f c=e5030380 d=2b4209f5 e=32ca2d8c f=0c657a79 g=9b27a401 h=714260ad
Étape 09: a=8c87346b b=8e04ecb9 c=85a07b5f d=e5030380 e=1cc92596 f=32ca